# Structuring a Model
In this notebook we do not add any new functionality to the model code, but instead work on improving the code itself.

In the first code block, we create a new type `Model` which is a `namedtuple` that contains the tensors relevant for our model. We could just use a regular tuple here, but the named tuple disambiguates the different entries to make the code more easily readable.

In [ ]:
import tensorflow as tf
import tensorflow.contrib.learn as tflearn
import numpy as np
from collections import namedtuple

NUM_EPISODES = 10
RESTORE = False

Model = namedtuple("Model", ["logits", "probabilities", "loss", "train_step"])

## Model Function
We have moved the code for building into its own function. For better readability separate logical parts of the model into their own functions.

### accuracy
This function calculates the accuracy of predictions. 
It puts all operations into a name scope to get a more readable graph (There is no need for a `VariableScope` here as there are no Variables). 
The `name_scope` function takes an optional second argument which is a list of tensors we want to use within this scope. Usually this can be omitted.

The calculation of the accuracy is done like this: First we calculate the prediced class (as an integer), and compare that to the given labels. This results in a boolean tensor, so before we calculate the mean we cast it to a floating point type.

In [ ]:
def accuracy(logits, labels, name="accuracy"):
    with tf.name_scope(name, [logits, labels]):
        predicted = tf.argmax(logits, axis=1, name="predicted")
        correct = tf.equal(tf.cast(predicted, tf.int32), labels, name="is_correct")
        return tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

### model_fn
We have put the code that builds the model graph inside its own function `model_fn`. We have kept the inputs `x` and `y` as parameters to that function. This separates the model definition from the way we provide inputs to that model. We can simply pass the placeholder tensors we have used until now and continue with the same behaviour as before, but we also could build a more advanced input processing and pass the resulting `x` and `y` tensors into the model function.

Since we return many different tensors, we do not use multiple return values in a tuple, but populate the `Model` namedtuple from above with the correct tensors.

We have also changed the creation of Variables to use `tf.get_variable` instead of `tf.Variable`and provide an initialization function to supply the initial value (`tf.initializers.random_uniform()` returns a function that is called within `get_variable`). Whereas `tf.get_variable` only becomes really important when coding functions that might be reused within models, it seems to be best practice to always use `tf.get_variable`.

In [ ]:
def model_fn(x, y):
    tf.summary.image("image", tf.reshape(x, (-1, 28, 28, 1)))
    W = tf.get_variable("W", shape=(784, 10), dtype=tf.float32, initializer=tf.initializers.random_uniform())
    b = tf.get_variable("b", shape=(10,), dtype=tf.float32, initializer=tf.initializers.random_uniform())
    l = tf.matmul(x, W) + b
    tf.summary.histogram("logits", l)
    p = tf.nn.softmax(l)
    tf.summary.histogram("probabilities", p)
    
    with tf.name_scope("loss_calculation"):
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y, depth=10), logits=l)
        loss = tf.reduce_mean(loss)
    
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy(l, y))

    optimizer = tf.train.GradientDescentOptimizer(0.1)
    global_step = tf.train.create_global_step()
    train_op = optimizer.minimize(loss, global_step=global_step)
    
    return Model(logits=l, probabilities=p, loss=loss, train_step=train_op)

# Graph Building
Here we build the model. We create a new `Graph` object, and use this as the default graph for the building. This way, we do not pollute the global default graph, and when we execute this cell multiple times we always get a new graph of the complete model, instead of adding to the global graph. 

We create the placeholders as before and apply `model_fn`, and also add the other "infrastructure" operations. Note in particular that the Saver also has to be created inside the `with` block as it adds operations to the Graph and needs access to the `tf.GraphKeys.GLOBAL_VARIABLES` collection to figure out what it is supposed to save.

In [ ]:
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32, (None, 784), name="x")
    y = tf.placeholder(tf.int32, (None), name="y")
    _, _, loss, train_op = model_fn(x, y)
    summaries = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

This is unchanged.

In [ ]:
# load the dataset and create a session
mnist = tflearn.datasets.load_dataset("mnist")
images = mnist.train.images
labels = mnist.train.labels

writer = tf.summary.FileWriter("structure_demo", graph)

## Training Loop
We have put the training loop inside a `tf.Session` with-block. This ensures that the session will be closed after this cell is executed. Since we no longer build the model in the global default graph, we need to explicitly pass the graph object to the newly created session. 

In [ ]:
with tf.Session(graph=graph) as session:
    if RESTORE:
        saver.restore(session, tf.train.latest_checkpoint("structure_demo"))
    else:
        init.run()
    
    for i in range(NUM_EPISODES):
        summary, _, step = session.run([summaries, train_op, tf.train.get_global_step()], {x: images, y: labels})
        writer.add_summary(summary, step)

    saver.save(session, "structure_demo/model", tf.train.global_step(session, tf.train.get_global_step()))
    writer.close()

## Graph Collections
For such a simple model, there is no need to manually work with graph collections. However, they provide the glue that is used to let many higher level tf facilities "magically" perform the right thing. We have used them indirectly for
 * handling the global step (GLOBAL_STEP)
 * the optimizer choice on which Variables to optimize (TRAINABLE_VARIABLES)
 * the Variables that are initialized by global_variables_initializer and save by Saver (GLOBAL_VARIABLES)
 * the Tensors merged by `summary.merge_all` (SUMMARIES)
 
 However, we can at least take a look at the values that have been automatically assinged to the collections. To get prettier printing, we extract the tensor names.

In [ ]:
collections = (tf.GraphKeys.GLOBAL_VARIABLES, tf.GraphKeys.TRAINABLE_VARIABLES, 
               tf.GraphKeys.SUMMARIES, tf.GraphKeys.GLOBAL_STEP)
for c in collections:
    vals = [str(t.name) for t in graph.get_collection(c)]
    print("'%s' = %s" % (c, vals))

## Closing Remarks
The separation in `model_fn` and input, graph collections etc. seem a bit overkill for a model this small, but become invaluable when models become more complex.